In [1]:
--- Libraries:
arguments = require 'Settings.arguments'
constants = require 'Settings.constants'
card_to_string = require 'Game.card_to_string_conversion'
card_tools = require 'Game.card_tools'
game_settings = require 'Settings.game_settings'
Plot = require 'itorch.Plot'
nninit = require 'nninit'
require 'torch'
require 'math'
require 'Tree.tree_builder'
require 'Tree.tree_data_generation'
require 'Tree.tree_visualiser'
require 'nn'
require 'Tree.tree_cfr'
require 'nngraph'
require 'optim'
require 'image'
require 'NN.strategy_net_builder'
require 'NN.strategy_net_trainer'
require 'Nn.StrategyCriterion'




In [2]:
--- Create the tree
builder = PokerTreeBuilder()
--- Parameters for the tree
params = {}
params.root_node = {}
params.root_node.board = card_to_string:string_to_board('')
params.root_node.street = 1
params.root_node.current_player = constants.players.P1
params.root_node.bets = arguments.Tensor{200, 200}
--- CFR Solver
starting_ranges = arguments.Tensor(constants.players_count, game_settings.card_count)
starting_ranges[1]:copy(card_tools:get_uniform_range(params.root_node.board))
starting_ranges[2]:copy(card_tools:get_uniform_range(params.root_node.board))

In [3]:
--- BUild trees
tree = builder:build_tree(params)
tree0 = builder:build_tree(params)
tree1 = builder:build_tree(params)

--- build data
game = TreeData(tree)
---game0 = TreeData(tree0)
---game1 = TreeData(tree1)
---game:get_training_set(tree,1)

In [4]:
tree_cfr = TreeCFR()
print("Solver")
tree_cfr:run_cfr(tree, starting_ranges,400)
print("getting training set")
game:get_training_set(tree,1)

Solver


getting training set


## Using the modules

In [5]:
-- Building the neural net model
strategy_nn = StrategyNN()
nn_model = strategy_nn.model
-- Building trainer:
nn_trainer = NNTrainer(game,nn_model)
-- Options:
opt = {}
opt.learningRate = 0.001
opt.momentum = 0.95
opt.validate = false

In [6]:
---current_criterion = nn.SmoothL1Criterion()
current_criterion = StrategyLoss()
train_loss,test_loss = nn_trainer:train(nn_model,current_criterion,opt,10000)

In [7]:
pred_strat = nn_model:forward({nn_trainer.all_data.features,nn_trainer.all_data.masks})
true_strat =nn_trainer.all_data.targets
print(torch.mean(torch.abs(pred_strat- true_strat)))

0.0026265346874302


In [8]:
--- Generating new initial strategies
game0 = TreeData(tree0)
game0:get_training_set(tree0,1)
game0:generate_new_initial_strategies(tree0,nn_trainer)

In [9]:
--- CFR:
tree_cfr0 = TreeCFR()
print("Solver")
tree_cfr0:run_cfr(tree0, starting_ranges,400)

Solver


In [10]:
--- CFR:
tree_cfr = TreeCFR()
print("Solver")
tree_cfr:run_cfr(tree, starting_ranges,400)

Solver


In [11]:
tree.strategy

 0.0003  0.0003  0.0003  0.0003  0.0044  0.0044
 0.4932  0.4932  0.9801  0.9801  0.8038  0.8038
 0.0869  0.0869  0.0165  0.0165  0.0151  0.0151
 0.4196  0.4196  0.0030  0.0030  0.1767  0.1767
[torch.FloatTensor of size 4x6]



In [12]:
tree0.strategy

 0.0006  0.0006  0.0006  0.0006  0.0089  0.0089
 0.4967  0.4967  0.9634  0.9634  0.8178  0.8178
 0.1242  0.1242  0.0299  0.0299  0.0216  0.0216
 0.3785  0.3785  0.0061  0.0061  0.1517  0.1517
[torch.FloatTensor of size 4x6]



In [13]:
function compare_two_trees(table1,table2)
    local difference = {}
    for i = 1, #table1 do
        difference[i] = torch.sqrt(torch.sum(torch.pow((table1[i] - table2[i]),2)))
    end
    return torch.FloatTensor(difference)
end

In [14]:
dif = compare_two_trees(tree_cfr.total_strategies,tree_cfr0.total_strategies)

In [15]:
local t2 = torch.range(1,dif:narrow(1,101,300):size(1))
local plot = Plot()

plot:line(t2,dif:narrow(1,101,300),'red' ,'train loss')
---plot:line(t2,test_loss1,'blue' ,'test loss')
plot:legend(true):title('Mean Square Error')
plot:draw()

In [108]:
function compare_two_trees2(table1,table2)
    local difference = {}
    for i = 500,#table2 do

        table.insert(difference, torch.sqrt(torch.sum(torch.pow((table1[i-400] - table2[i]),2))))
---        print(difference[i])    
    end
    return torch.FloatTensor(difference)
end

In [109]:
diff = compare_two_trees2(tree_cfr.total_strategies,tree_cfr1.total_strategies)
diff_nn = compare_two_trees2(tree_cfr0.total_strategies,tree_cfr1.total_strategies)

In [134]:
t2 = torch.range(2,diff:narrow(1,10,201):size(1))
local plot = Plot()

plot:line(t2,diff:narrow(1,10,200),'red' ,'S0: Strategy initialized with target values')
plot:line(t2,diff_nn:narrow(1,10,200),'blue' ,'SNN : Strategy initialized with NN')
plot:legend(true):title('MSE between optimal strategies and S0/SNN ')
plot:draw()

In [132]:
t2:size()

 199
[torch.LongStorage of size 1]



In [133]:
diff:narrow(1,10,200):size()

 200
[torch.LongStorage of size 1]



In [ ]:
--- CFR:
tree_cfr1 = TreeCFR()
print("Solver")
tree_cfr1:run_cfr(tree, starting_ranges,400)

In [76]:
#tree_cfr0.total_strategies

400


In [72]:
#tree_cfr1.total_strategies

800


In [73]:
#tree_cfr.total_strategies

400


In [ ]:
tree3 = builder:build_tree(params)


In [ ]:
game3 = TreeData(tree3)
game3:get_training_set(tree3,1)
game3:generate_new_initial_strategies(tree3,nn_trainer)

In [ ]:
local starting_ranges = arguments.Tensor(constants.players_count, game_settings.card_count)
starting_ranges[1]:copy(card_tools:get_uniform_range(params.root_node.board))
starting_ranges[2]:copy(card_tools:get_uniform_range(params.root_node.board))
tree_cfr0 = TreeCFR()
tree_cfr0:run_cfr(tree0, starting_ranges,300)

In [ ]:
tree2 = builder:build_tree(params)
--- CFR Solver
local starting_ranges = arguments.Tensor(constants.players_count, game_settings.card_count)
starting_ranges[1]:copy(card_tools:get_uniform_range(params.root_node.board))
starting_ranges[2]:copy(card_tools:get_uniform_range(params.root_node.board))
tree_cfr2 = TreeCFR()
print("Solver")
tree_cfr2:run_cfr(tree2, starting_ranges,300)
---tree_cfr:run_cfr(tree0, starting_ranges,50)

In [ ]:
tree0.strategy

In [ ]:
tree2.strategy

In [ ]:
tree.strategy

In [ ]:
print(torch.sum(tree0.children[2].regrets)/300)
print(torch.sum(tree.children[2].regrets)/500)
print(torch.sum(tree2.children[2].regrets)/300)

In [ ]:
local new_model = nn_trainer.set_new_model()
local current_criterion = nn.SmoothL1Criterion()

-- Options:
local opt = {}
opt.learningRate = 0.001
opt.momentum = 0.95
opt.validate = true

train_loss1,test_loss1 = nn_trainer:train(new_model,current_criterion,opt,10000)

In [ ]:
local t2 = torch.range(1,train_loss1:size(1))
local plot = Plot()

plot:line(t2,train_loss1,'red' ,'train loss')
plot:line(t2,test_loss1,'blue' ,'test loss')
plot:legend(true):title('Mean Square Error')
plot:draw()